# Random Acts of Pizza

![Pizza Badges](../reports/images/pizzas.png)

---

**Auteur**: Akim van Eersel  
**Date**: 2022-12-30

# Predire la donation de pizza à travers un subreddit dédié

## Subreddit RAOP

Le r/RAOP permet d'obtenir des données sur des demandes faites par des internautes afin de potentiellement se voir offrir une pizza.  
Ces données se composent tout ce qui se raccorde à la demande (contenu et métadonnées) ainsi qu'à certaines informations du profil Reddit du demandeur.

## Hypothèses

**Localisation** : d'un point de vue business, l'utilisation de données provenant de Reddit ne doit pas influer sur la population ciblée.  
**Décalage temporel** : les données datent d'il y a 9 ans et se sont écoulées sur une période de 3 ans. Le fait de faire une donation et ce relevant d'une pizza est jugé intemporel socialement et économiquement.  
**Sagesse des foules** : la donation d'un-e individu à un-e autre est déterminée comme étant une décision impartielement légitime avec un petit delta du au procédé global.

## Enjeux business

L'idée de pouvoir prédire la donation de pizza est projeté sur une étude de cas où une chaîne de pizza cherche à mettre en place un procédé similaire pour gagner entre autres en notoriété de marque.  
Les enjeux sont alors :  
1. d'obtenir un modèle pouvant prédire la légitimité d'une demande par un-e individu au moment de la demande,  
2. de pouvoir valider cette prédiction a posteriori pour appuyer en aval la décision d'offrir une pizza de la part du restaurant.

### Décision applicative

Le modèle décrit ne devrait pas être l'unique décisionnaire sur la décision finale de donation, surtout si ce dernier n'est pas ultra-performant. Celui-ci devrait être réutilisé comme une des entrées pour un second algorithme de décision, et en imaginant facilement d'autre variables liées aux implications business et de supply chain.

In [1]:
import string

import numpy as np
import pandas as pd
from datetime import datetime as dt
from scipy.stats import f_oneway
import seaborn as sns

from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import train_test_split, HalvingGridSearchCV
from sklearn.feature_selection import mutual_info_classif, chi2, SelectKBest
from sklearn.decomposition import TruncatedSVD, NMF
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, f1_score, precision_score, confusion_matrix
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer, MaxAbsScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB
import xgboost as xgb

from nltk.corpus import stopwords
from nltk import word_tokenize

ModuleNotFoundError: No module named 'xgboost'

## Présentation des données

In [3]:
pizza_raw_data = pd.read_json('../data/pizza_data.json',
                              dtype={"giver_username_if_known": str,
                                     "number_of_upvotes_of_request_at_retrieval": int,
                                     "post_was_edited": bool,
                                     "request_id": str,
                                     "request_number_of_comments_at_retrieval": int,
                                     "request_text": str,
                                     "request_text_edit_aware": str,
                                     "request_title": str,
                                     "requester_account_age_in_days_at_request": float,
                                     "requester_account_age_in_days_at_retrieval": float,
                                     "requester_days_since_first_post_on_raop_at_request": float,
                                     "requester_days_since_first_post_on_raop_at_retrieval": float,
                                     "requester_number_of_comments_at_request": int,
                                     "requester_number_of_comments_at_retrieval": int,
                                     "requester_number_of_comments_in_raop_at_request": int,
                                     "requester_number_of_comments_in_raop_at_retrieval": int,
                                     "requester_number_of_posts_at_request": int,
                                     "requester_number_of_posts_at_retrieval": int,
                                     "requester_number_of_posts_on_raop_at_request": int,
                                     "requester_number_of_posts_on_raop_at_retrieval": int,
                                     "requester_number_of_subreddits_at_request": int,
                                     "requester_received_pizza": bool,
                                     "requester_subreddits_at_request": list,
                                     "requester_upvotes_minus_downvotes_at_request": int,
                                     "requester_upvotes_minus_downvotes_at_retrieval": int,
                                     "requester_upvotes_plus_downvotes_at_request": int,
                                     "requester_upvotes_plus_downvotes_at_retrieval": int,
                                     "requester_user_flair": str,
                                     "requester_username": str,
                                     "unix_timestamp_of_request": int,
                                     "unix_timestamp_of_request_utc": int})

pizza_prevented_data = pizza_raw_data.loc[:, ~(pizza_raw_data
                                               .columns
                                               .isin(["giver_username_if_known",
                                                      "requester_user_flair",
                                                      "request_text",
                                                      "post_was_edited"]))
                       ]

dataset_period = (dt.strptime("29/09/2013", "%d/%m/%Y") - dt.strptime("08/12/2010", "%d/%m/%Y")).days

target_name = 'requester_received_pizza'
seed = 101
X = pizza_prevented_data.copy()
y = X.pop(target_name)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=seed, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=seed, stratify=y_test)

print(f'''From our {pizza_raw_data.shape[0]} samples, we'll use {X_train.shape[0]} of them to train the model.
From {len(y_train)} requests, {y_train.sum()} led to a donation, which represents {round(y_train.sum()/len(y_train), 3)*100}%.
Thus, among {dataset_period} days, 1 pizza was donate every {round(dataset_period/y_train.sum(), 2)} day.''')

From our 4040 samples, we'll use 3636 of them to train the model.
From 3636 requests, 895 led to a donation, which represents 24.6%.
Thus, among 1026 days, 1 pizza was donate every 1.15 day.
